In [1]:
import random
import math
import numpy as np
from scipy import special
from scipy.stats import genhyperbolic
from scipy.stats import johnsonsu
from scipy.stats import dgamma
from scipy.stats import chi2
import time

In [2]:
def log(s):
  if LOG: print(s)

Condiciones Iniciales

In [3]:
### Variables globales ###
LOG = False
HV = 999999999

# Variables de control. Se inicializan manualmente
CCS, CCM, UCE, CCE = 0, 0, 0, 0
CCMAux = 0

T, TF = 0, 0

CCER, CA , NAE = 0, 0, 0
NT, NTARB, NTARM, NTARA = 0, 0, 0, 0
NAARM = 0,


In [4]:
def condicionesIniciales(t, tf):
  global T, TF
  global CCS, CCM, UCE, CCE
  global CCMAux
  global CCER, CA , NAE
  global NT, NTARB, NTARM, NTARA
  global NAARM


  T = t
  TF = tf

  #Acumuladores y auxiliares
  CCMAux = 0
  CCER = 0
  CA = 0
  #Accidentes
  NAE = 0
  NT = 0
  NTARB = 0
  NTARM = 0
  NTARA = 0
  NAARM = 0


In [5]:
def getCAPSC(): #Desde 2015
  p = 3.7610237363153907
  a = 0.00209573078650936
  b = 0.00029747269395355913
  loc = 41.82717215116361
  scale = 0.011524812451895364

  R = random.uniform(0, 1)
  return abs(math.ceil(genhyperbolic.ppf(R, p, a, b, loc, scale)))

In [6]:
def getCAPSM(): #Desde 2015
  a = 0.366308266363123
  b = 1.2863317264123912
  loc = 79.77858636796023
  scale = 13.852446969708332

  R = random.uniform(0, 1)
  return abs(math.ceil(johnsonsu.ppf(R, a, b, loc, scale)))

In [7]:
def getNSR():
  a = 2.50208624966522,
  loc = 4.390091400881478,
  scale = 0.6444600118419814

  R = random.uniform(0, 1)
  return abs(math.ceil(dgamma.ppf(R, a, loc, scale)))

In [8]:
def seEvitaAccidente():
  # se evita un accidente si el Random es Menor al factor F
  f = ((1-0.12)**CCMAux)*((1-0.06)**CCS)
  R = random.uniform(0, 1)
  if (R <= f):
    return False
  else:
    return True

In [9]:
def calcularResultados():

  print(f"Fin de la simulacion en T : {T} seg")
  print(f"Total de Accidentes: {NT}")
  print(f"Total Accidentes Evitados: {NAE}")
  print("Variables de control")
  print(f"CCS={CCS}, CCM={CCM}, UCE={UCE}, CCE={CCE}")

  print("\n")
  PARA = NTARA * 100 / NT
  print(f"Porcentaje de Accidentes de Riesgo Alto: {PARA}%")

  print("\n")
  CPA = CA/NT
  print(f"Costo promedio de Accidentes: ${CPA}")

  print("\n")
  CPMARB = NTARB * 4 / T
  CPMARM = NTARM * 4 / T
  CPMARA = NTARA * 4 / T
  print(f"Cantidad Promedio Mensual de Accidentes de Riesgo Bajo: {CPMARB}")
  print(f"Cantidad Promedio Mensual de Accidentes de Riesgo Medio: {CPMARM}")
  print(f"Cantidad Promedio Mensual de Accidentes de Riesgo Alto: {CPMARA}")

  print("\n")
  PAE = NAE*100/NT
  print(f"Porcentaje de Accidentes Evitados: {PAE}")

  print("\n")
  PCER = CCER*100/(T*CCM/4)
  print(f"Porcentaje de Capacitaciones Extra Realizadas: {PCER}")

  return

In [10]:
def simular():
  global T, TF
  global CCS, CCM, UCE, CCE
  global CCMAux
  global CCER, CA , NAE
  global NT, NTARB, NTARM, NTARA
  global NAARM

  if (CCM < 0 or CCS < 0 or CCE < 0 or UCE < 0 ):
    print("El valor de las variables de control debe ser mayor a 0")
    print(f"CCM={CCM}, CCS={CCS}, CCE={CCE}, UCE={UCE}")
    return

  if (TF <= T):
    print("El valor de TF debe ser mayor que T")
    print(f"T={T}, TF={TF}")
    return

  if (T < 0 or TF < 0):
    print("El valor de T y TF debe ser mayor a 0")
    print(f"T={T}, TF={TF}")
    return

  print(f"Inicio de simulacion: T={T}, TF={TF}")

  while (True):
    T+=1
    if(T % 4 == 0):
      if(NAARM >= UCE):
        CCMAux = CCM + CCE
        CCER += CCE
      else:
        CCMAux = CCM
      CA += 6000000*CCMAux
      NAARM = 0;

    CA += 1000000*CCS
    CAPSC = getCAPSC()
    CAPSM = getCAPSM()

    for i in range(CAPSC + CAPSM):
      if (seEvitaAccidente()):
        NAE += 1
        NT += 1
      else:
        NT += 1
        NSR = getNSR()
        if(NSR == 0 or NSR == 1 or NSR == 2 or NSR == 3):
          NTARB += 1
        elif(NSR == 4 or NSR ==5 or NSR ==6 or NSR ==7):
          NTARM += 1
        else:
          NTARA += 1
          NAARM +=1
        CA += 100000 + 200000*(2**(NSR - 1))

    if (T < TF):
      continue
    else:
      break

  calcularResultados()

  return

In [11]:
# Para correr la simulacion
  # 1. ejecutar todas las secciones de codigo desde arriba hacia abajo. Cuando termina se indica con un tilde verde que el codigo ya esta cargado en memoria y tambien variables
  # 2. darle el valor que quieran a las variables de control y de costos
  # 3. ejecutar este bloque de codigo

global CCS, CCM, UCE, CCE

  # Variables de control
CCS = 2      # cantidad de Controles semanales
CCM = 7       # Cantidad de Capacitaciones mensuales
UCE = 16      # Umbral realización capacitaciones extra
CCE = 2       # Cantidad de Capacitaciones extra

# Parametros de configuracion
LOG = True   # activa los mensajes de log para debug

  # el tiempo esta en segundos
condicionesIniciales(t = 0, tf = 1040)  # 20 años aprox
simular()

Inicio de simulacion: T=0, TF=1040


/tmp/ipython-input-1477690184.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return abs(math.ceil(dgamma.ppf(R, a, loc, scale)))


Fin de la simulacion en T : 1040 seg
Total de Accidentes: 127914
Total Accidentes Evitados: 83812
Variables de control
CCS=2, CCM=7, UCE=16, CCE=2


Porcentaje de Accidentes de Riesgo Alto: 2.6361461607017214%


Costo promedio de Accidentes: $2694599.4965367354


Cantidad Promedio Mensual de Accidentes de Riesgo Bajo: 42.926923076923075
Cantidad Promedio Mensual de Accidentes de Riesgo Medio: 113.72692307692307
Cantidad Promedio Mensual de Accidentes de Riesgo Alto: 12.96923076923077


Porcentaje de Accidentes Evitados: 65.52214769298122


Porcentaje de Capacitaciones Extra Realizadas: 6.923076923076923
